https://github.com/algo2t/alphatrade

In [1]:
from omspy_brokers.sasonline import Sasonline
from toolkit.fileutils import Fileutils
from toolkit.utilities import Utilities
from toolkit.logger import Logger

In [4]:
import pathlib
p = pathlib.Path().resolve()
fpath = p.parents[0]
fpath

WindowsPath('C:/Users/dell/Downloads/python_pet_projects/sas_algo/sas-order')

In [6]:
import os 
os.path.exists(fpath / "sas.yaml")

True

In [7]:
cred = Fileutils().get_lst_fm_yml(fpath / "sas.yaml")
print(cred)

{'login_id': 'HW105', 'password': 'Ad04@now', 'totp': 'SVT7OUVZ4KGBE7T5'}


In [8]:
sas = Sasonline(cred['login_id'], cred['password'], cred['totp'])
if sas.authenticate():
    print("\n successfully authenticated")

{'pan_number': 'AAYPW5593J', 'name': 'HEMANT DINESH WAIRAGADE', 'login_id': 'HW105', 'exchanges': ['BSE', 'CDS', 'NSE', 'NFO'], 'email_address': 'hemant271172@gmail.com', 'dp_ids': [], 'broker_name': 'SOUTHASIAN', 'banks': [{'branch_name': '', 'bank_name': 'HDFC Bank', 'account_id': 'HW105'}, {'branch_name': '', 'bank_name': '', 'account_id': 'HW105'}, {'branch_name': '', 'bank_name': '', 'account_id': 'HW105'}, {'branch_name': '', 'bank_name': '', 'account_id': 'HW105'}, {'branch_name': '', 'bank_name': '', 'account_id': 'HW105'}, {'branch_name': '', 'bank_name': '', 'account_id': 'HW105'}], 'backoffice_enabled': True}

 successfully authenticated


In [9]:
import configparser
parser = configparser.ConfigParser()
parser.read('config.ini')
configuration_details = dict(parser.items("DEFAULT"))
sl_percentage = float(configuration_details["sl_percentage"])
profit_percentage = float(configuration_details["profit_percentage"])
threshold_percentage = float(configuration_details["threshold_percentage"])
is_move_stop_hard = bool(int(configuration_details["is_move_stop_hard"]))

{'sl_percentage': '3.0', 'profit_percentage': '7.0'}

In [12]:
sas_orders = sas.orders
sas_orders

{'status': 'success',
 'message': '',
 'data': {'pending_orders': [],
  'completed_orders': [{'validity': 'DAY',
    'user_order_id': '1',
    'trigger_price': 0.0,
    'transaction_type': 'BUY',
    'trading_symbol': 'NIFTY23JUN18700CE',
    'remaining_quantity': 200,
    'rejection_reason': '--',
    'quantity': 200,
    'product': 'MIS',
    'price': 101.85,
    'order_type': 'LIMIT',
    'order_tag': '',
    'order_status': 'cancelled',
    'order_entry_time': 1687855898,
    'oms_order_id': '230627000180764',
    'nest_request_id': '1',
    'lotsize': 50,
    'login_id': 'HW105',
    'leg_order_indicator': '',
    'instrument_token': '60233',
    'filled_quantity': 0,
    'exchange_time': 1687855899,
    'exchange_order_id': '1000000058009338',
    'exchange': 'NFO',
    'disclosed_quantity': 0,
    'client_id': 'HW105',
    'average_price': 0.0},
   {'validity': 'DAY',
    'user_order_id': '2',
    'trigger_price': 0.0,
    'transaction_type': 'SELL',
    'trading_symbol': 'NIFTY

In [ ]:
import enum

class TransactionType(enum.Enum):
    Buy = 'BUY'
    Sell = 'SELL'


class OrderType(enum.Enum):
    Market = 'MARKET'
    Limit = 'LIMIT'
    StopLossLimit = 'SL'
    StopLossMarket = 'SL-M'


class ProductType(enum.Enum):
    Intraday = 'I'
    Delivery = 'D'
    CoverOrder = 'CO'
    BracketOrder = 'BO'

In [ ]:

order_tracker = []
def track_orders():
    global order_tracker
    for order in sas_orders["data"]["completed_orders"]:
        if order["order_status"] == "complete":
            for tracked_order in order_tracker:
                if order["oms_order_id"] in tracked_order["algo_placed_order_id"]:
                    print("order complete for algo placed order. Squaring off")
                    tracked_order["square_off"] = 1
                    if sum(1 
                           for d in order_tracker for k, v in d.items() 
                           if (k == "trading_symbol" and v == order["instrument_token"] and "square_off" in d) 
                           ) == 1:
                        sas.broker.unsubscribe(sas.broker.get_instrument_by_symbol(order["exchange"], order["instrument_token"]), 1)
                    break
            else:
                def place_new_sl_order():
                    # Place 3% SL order on receipt of order from trader
                    args = {
                        "transaction_type" : TransactionType.Sell,
                        "instrument": sas.broker.get_instrument_by_token(order["exchange"], order["instrument_token"]), # exchange will always be NfO?
                        "quantity": order["quantity"],
                        "order_type": OrderType.StopLossMarket,
                        "product_type": order["product"],
                        "price": 0.0,
                        "trigger_price": order["price"] * (1 - sl_percentage / 100)
                    }
                    status = sas.order_place(**args)
                    return status['data']['oms_order_id']
                
                new_entry_to_tracker = {}
                new_entry_to_tracker["algo_placed_order_id"] = place_new_sl_order()
                new_entry_to_tracker["trader_placed_order_id"] = order["oms_order_id"]
                new_entry_to_tracker["price"] = float(order["price"])
                new_entry_to_tracker["quantity"] = int(order["quantity"])
                new_entry_to_tracker["trading_symbol"] = order["trading_symbol"]
                new_entry_to_tracker["instrument_token"] = order["instrument_token"]
                new_entry_to_tracker["exchange"] = order["exchange"]
                new_entry_to_tracker["product"] = order["product"]
                
                ## unsubscribe 
                sas.broker.subscribe(sas.broker.get_instrument_by_symbol(order["exchange"], order["instrument_token"]), 1)
                order_tracker.append(new_entry_to_tracker)
                
                
            print(order)

In [ ]:
socket_opened = False
ltp_symbol_dict = {}
def event_handler_quote_update(message):
    global ltp_symbol_dict, order_tracker
    print(f"quote update {message}")

    symbol = message["trade_symbol"]
    ltp = message["ltp"]
    ltp_symbol_dict[symbol] = ltp
    for order in order_tracker:
        if order["trading_symbol"] == symbol:
            if ltp < order["price"] * (1 - sl_percentage / 100):
                # SL limit has reached
                # manually do something?
                pass
            elif order["price"] * (1 + threshold_percentage / 100) > ltp > order["price"] * (1 + profit_percentage / 100) and is_move_stop_hard:
                args = {
                        "transaction_type" : TransactionType.Sell,
                        "instrument": sas.broker.get_instrument_by_token(order["exchange"], order["instrument_token"]), # exchange will always be NfO?
                        "quantity": order["quantity"],
                        "order_type": OrderType.StopLossMarket,
                        "product_type": order["product"],
                        "price": 0.0,
                        "trigger_price": order["price"] * (1 + threshold_percentage / 100)
                    }
                _ = sas.order_modify(**args)
            elif ltp >= order["price"] * (1 + profit_percentage / 100):
                args = {
                        "transaction_type" : TransactionType.Sell,
                        "instrument": sas.broker.get_instrument_by_token(order["exchange"], order["instrument_token"]), # exchange will always be NfO?
                        "quantity": order["quantity"],
                        "order_type": OrderType.StopLossMarket,
                        "product_type": order["product"],
                        "price": 0.0,
                        "trigger_price": order["price"] * (1 + profit_percentage / 100)
                    }
                _ = sas.order_modify(**args)
                order["square_off"] = 1
                if sum(1 
                        for d in order_tracker for k, v in d.items() 
                        if (k == "trading_symbol" and v == order["instrument_token"] and "square_off" not in d) 
                        ) == 1:
                    sas.broker.unsubscribe(sas.broker.get_instrument_by_symbol(order["exchange"], order["instrument_token"]), 1)
    

def open_callback():
    global socket_opened
    socket_opened = True


def run_statergy():
    sas.broker.start_websocket(subscribe_callback=event_handler_quote_update,
                        socket_open_callback=open_callback,
                        run_in_background=True)
    while(socket_opened==False):
        pass
    while True:
        track_orders()
